# Metadata fixing

In [1]:
import pandas as pd

In [2]:
md_full_fp = '../metadata/eco_md-qiime_host_species_eco_metadata_by_SampleID_gut_11.28.18.txt'
md_5per_fp = '../metadata/5per.11.28.18.short.txt'
md_5per_10k_fp = '../metadata/5per_10k.11.28.18.short.txt'


In [3]:
md_full = pd.read_csv(md_full_fp, sep='\t')
md_5per = pd.read_csv(md_5per_fp, sep='\t')
md_5per_10k = pd.read_csv(md_5per_10k_fp, sep='\t')

/home/jgsanders/.conda/envs/jupyter-qiime2-2019.1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (34,35,83,168) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
md_full.head()

,SampleID,studyID,Orig_ID,BarcodeSequence,LinkerPrimerSequence,extracted,sequenced_se,sequenced_pe,lane_primerplate,lane,...,LAV.ln_caelen.estimate,LAV.ln_sivol.estimate,LAV.ln_nsa.estimate,LAV.ln_mass.imputed,LAV.ln_silen.imputed,LAV.ln_lilen.imputed,LAV.ln_stomas.imputed,LAV.ln_caelen.imputed,LAV.ln_sivol.imputed,LAV.ln_nsa.imputed
0,11212.AC622,11212,AC622.blackhowler,NNNN,GTGTGCCAGCMGCCGCGGTAA,yes,yes,no,NaN,NaN,...,2.389167,5.551645,6.708361,True,True,True,True,True,True,True
1,11212.AC668,11212,AC668.blackhowler,NNNN,GTGTGCCAGCMGCCGCGGTAA,yes,yes,no,NaN,NaN,...,2.389167,5.551645,6.708361,True,True,True,True,True,True,True
2,11212.AC673,11212,AC673.blackhowler,NNNN,GTGTGCCAGCMGCCGCGGTAA,yes,yes,no,NaN,NaN,...,2.389167,5.551645,6.708361,True,True,True,True,True,True,True
3,11212.AC674,11212,AC674.blackhowler,NNNN,GTGTGCCAGCMGCCGCGGTAA,yes,yes,no,NaN,NaN,...,2.389167,5.551645,6.708361,True,True,True,True,True,True,True
4,11212.AC679,11212,AC679.blackhowler,NNNN,GTGTGCCAGCMGCCGCGGTAA,yes,yes,no,NaN,NaN,...,2.389167,5.551645,6.708361,True,True,True,True,True,True,True


## Fixes

In [5]:
md_full.loc[md_full['Taxonomy_Order'] == 'Cetacea', 'Taxonomy_Family']

6075         Balaenidae
6076         Balaenidae
6077         Balaenidae
6078         Balaenidae
6079         Balaenidae
6080         Balaenidae
6081         Balaenidae
6082    Balaenopteridae
6083    Balaenopteridae
6084    Balaenopteridae
6085    Balaenopteridae
6086        Delphinidae
6087        Delphinidae
6088       Monodontidae
6091        Delphinidae
6100    Balaenopteridae
Name: Taxonomy_Family, dtype: object

Cetacea -> cetartiodactyla

In [6]:
md_full.loc[md_full['Taxonomy_Order'] == 'Cetacea', 'Taxonomy_Order'] = 'Cetartiodactyla'
md_5per.loc[md_5per['Taxonomy_Order'] == 'Cetacea', 'Taxonomy_Order'] = 'Cetartiodactyla'
md_5per_10k.loc[md_5per_10k['Taxonomy_Order'] == 'Cetacea', 'Taxonomy_Order'] = 'Cetartiodactyla'

Chinese water dragons -> not turtles

In [7]:
md_full.loc[md_full['TimeTree_returned'] == 'Physignathus_cocincinus', 'Taxonomy_Class'] = 'Lepidosauria'
md_5per.loc[md_5per['TimeTree_returned'] == 'Physignathus_cocincinus', 'Taxonomy_Class'] = 'Lepidosauria'
md_5per_10k.loc[md_5per_10k['TimeTree_returned'] == 'Physignathus_cocincinus', 'Taxonomy_Class'] = 'Lepidosauria'

African Spurred Tortois -> not lizard

In [8]:
md_full.loc[md_full['TimeTree_returned'] == 'Centrochelys_sulcata', 'Taxonomy_Class'] = 'Chelonia'
md_5per.loc[md_5per['TimeTree_returned'] == 'Centrochelys_sulcata', 'Taxonomy_Class'] = 'Chelonia'
md_5per_10k.loc[md_5per_10k['TimeTree_returned'] == 'Centrochelys_sulcata', 'Taxonomy_Class'] = 'Chelonia'

## Migration metadata

From Kevin Winker

In [9]:
migrate_md_fp = '../metadata/bird_species_unique_with migratory status.xlsx'

migrate_md = pd.read_excel(migrate_md_fp, skipfooter=12)

In [10]:
set(migrate_md['Taxonomy_Species']) - set(md_full['Taxonomy_Species'])

{'Caracara plancus',
 'Cathartes burrovianus',
 'Cathartes melambrotus',
 'Laniurus atrococcineus',
 'Milvago chimachima'}

In [11]:
 set(md_full.loc[md_full['Taxonomy_Species'] == 'Aves', 
                 'NCBI_canonical_form']) - set(migrate_md['Taxonomy_Species']) 

set()

Remove duplicates, which are always second row:

In [12]:
migrate_md_nodup = migrate_md.groupby('Taxonomy_Species').first().reset_index()

merge with metadata

In [13]:
migrate_md_nodup.columns

Index(['Taxonomy_Species', 'Corrected Species name', 'migratory',
       'Citation for migratory status', 'Notes', 'NCBI_name_string',
       'NCBI_canonical_form', 'Taxonomy_Class', 'Taxonomy_Order',
       'Taxonomy_Family', 'Taxonomy_Genus', 'Taxonomy_Subspecies',
       'migratory.1'],
      dtype='object')

In [14]:
'Corrected Species name' in md_full.columns

False

Check that we don't have any duplicates remaining

In [15]:
migrate_md_nodup['Taxonomy_Species'].value_counts()[0]

1

In [16]:
migrate_merge  = migrate_md_nodup[['Taxonomy_Species',
                                   'Corrected Species name', 
                                   'migratory',
                                   'Citation for migratory status', 
                                   'Notes']]

migrate_merge.columns = ['Taxonomy_Species',
                        'Winker Corrected Species name', 
                        'migratory',
                        'Citation for migratory status', 
                        'Migration Notes']

In [17]:
md_full = pd.merge(left=md_full, 
                   right=migrate_merge,
                   on='Taxonomy_Species',
                   how='left') 

In [18]:
md_5per = pd.merge(left=md_5per, 
                   right=migrate_merge,
                   on='Taxonomy_Species',
                   how='left')

In [19]:
md_5per_10k = pd.merge(left=md_5per_10k, 
                       right=migrate_merge,
                       on='Taxonomy_Species',
                       how='left')

## Write new files

In [20]:
new_md_full_fp = '../metadata/eco_md-qiime_host_species_eco_metadata_by_SampleID_gut_2.18.19.txt'
new_md_5per_fp = '../metadata/5per.2.18.19.short.txt'
new_md_5per_10k_fp = '../metadata/5per_10k.2.18.19.short.txt'

In [21]:
md_full.to_csv(new_md_full_fp, sep='\t', index=False)
md_5per.to_csv(new_md_5per_fp, sep='\t', index=False)
md_5per_10k.to_csv(new_md_5per_10k_fp, sep='\t', index=False)